In [1]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.7 MB/s 
     |████████████████████████████████| 895 kB 42.7 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 6.5 MB 36.2 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import pipeline

In [3]:
nlp = pipeline('fill-mask')

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [4]:
nlp.tokenizer.mask_token

'<mask>'

In [5]:
f'pizza is my {nlp.tokenizer.mask_token}'

'pizza is my <mask>'

In [18]:
nlp(f'pizza is my {nlp.tokenizer.mask_token}')

[{'score': 0.07185398787260056,
  'sequence': 'pizza is my favorite',
  'token': 2674,
  'token_str': ' favorite'},
 {'score': 0.036543797701597214,
  'sequence': 'pizza is my jam',
  'token': 11914,
  'token_str': ' jam'},
 {'score': 0.03593211621046066,
  'sequence': 'pizza is my favourite',
  'token': 5548,
  'token_str': ' favourite'},
 {'score': 0.029360603541135788,
  'sequence': 'pizza is my cake',
  'token': 8492,
  'token_str': ' cake'},
 {'score': 0.02655790001153946,
  'sequence': 'pizza is my lunch',
  'token': 4592,
  'token_str': ' lunch'}]

In [19]:
nlp(f'pizza is {nlp.tokenizer.mask_token} my {nlp.tokenizer.mask_token} food.')

[[{'score': 0.08988059312105179,
   'sequence': '<s>pizza is probably my<mask> food.</s>',
   'token': 1153,
   'token_str': ' probably'},
  {'score': 0.07740215212106705,
   'sequence': '<s>pizza is definitely my<mask> food.</s>',
   'token': 2299,
   'token_str': ' definitely'},
  {'score': 0.05823001265525818,
   'sequence': '<s>pizza is just my<mask> food.</s>',
   'token': 95,
   'token_str': ' just'},
  {'score': 0.05166766420006752,
   'sequence': '<s>pizza is now my<mask> food.</s>',
   'token': 122,
   'token_str': ' now'},
  {'score': 0.04698041453957558,
   'sequence': '<s>pizza is always my<mask> food.</s>',
   'token': 460,
   'token_str': ' always'}],
 [{'score': 0.5955325365066528,
   'sequence': '<s>pizza is<mask> my favorite food.</s>',
   'token': 2674,
   'token_str': ' favorite'},
  {'score': 0.1784542202949524,
   'sequence': '<s>pizza is<mask> my comfort food.</s>',
   'token': 5863,
   'token_str': ' comfort'},
  {'score': 0.11437921971082687,
   'sequence': '<s>

In [6]:
from transformers import TFAutoModelForMaskedLM, AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
model = TFAutoModelForMaskedLM.from_pretrained("distilroberta-base")

Downloading:   0%|          | 0.00/465M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


In [8]:
vocab = tokenizer.get_vocab()
id2word = {i: word for word, i in vocab.items()}

In [29]:
sequence = f'pizza is my {tokenizer.mask_token} food.'
sequence

'pizza is my <mask> food.'

In [30]:
input_ids = tokenizer.encode(sequence,return_tensors='tf')

In [11]:
tokenizer.mask_token_id

50264

In [31]:
input_ids[0]

<tf.Tensor: shape=(9,), dtype=int32, numpy=
array([    0,   642, 35280,    16,   127, 50264,   689,     4,     2],
      dtype=int32)>

In [13]:
input_ids[0] == tokenizer.mask_token_id

<tf.Tensor: shape=(9,), dtype=bool, numpy=array([False, False, False, False, False,  True, False, False, False])>

In [16]:
import tensorflow as tf
mask_token_indices = tf.where(input_ids[0]==tokenizer.mask_token_id)[0].numpy().tolist()
mask_token_indices

[5]

In [21]:
result = model(input_ids)
result

TFMaskedLMOutput([('logits',
                   <tf.Tensor: shape=(1, 9, 50265), dtype=float32, numpy=
                   array([[[35.21495   , -3.9780874 , 18.639315  , ...,  2.8239052 ,
                             5.499281  , 12.741692  ],
                           [ 0.24456188, -5.0084376 , 12.591685  , ..., -2.447715  ,
                            -2.577407  , -0.09171152],
                           [ 1.9546525 , -4.852409  ,  5.4015346 , ..., -3.9360301 ,
                            -1.5068203 , -0.11707431],
                           ...,
                           [-3.2526023 , -6.279535  ,  5.785829  , ..., -2.06605   ,
                            -1.8451253 , -2.678122  ],
                           [18.32295   , -5.372933  , 19.210497  , ..., -0.04520693,
                             2.2998261 ,  4.34124   ],
                           [ 9.742845  , -6.0795035 , 26.81919   , ..., -1.9886105 ,
                            -4.526572  ,  4.77639   ]]], dtype=float32)>)])

In [46]:
result.logits

<tf.Tensor: shape=(1, 9, 50265), dtype=float32, numpy=
array([[[35.21495   , -3.9780874 , 18.639315  , ...,  2.8239052 ,
          5.499281  , 12.741692  ],
        [ 0.24456188, -5.0084376 , 12.591685  , ..., -2.447715  ,
         -2.577407  , -0.09171152],
        [ 1.9546525 , -4.852409  ,  5.4015346 , ..., -3.9360301 ,
         -1.5068203 , -0.11707431],
        ...,
        [-3.2526023 , -6.279535  ,  5.785829  , ..., -2.06605   ,
         -1.8451253 , -2.678122  ],
        [18.32295   , -5.372933  , 19.210497  , ..., -0.04520693,
          2.2998261 ,  4.34124   ],
        [ 9.742845  , -6.0795035 , 26.81919   , ..., -1.9886105 ,
         -4.526572  ,  4.77639   ]]], dtype=float32)>

In [47]:
logits = result[0]
logits

<tf.Tensor: shape=(1, 9, 50265), dtype=float32, numpy=
array([[[35.21495   , -3.9780874 , 18.639315  , ...,  2.8239052 ,
          5.499281  , 12.741692  ],
        [ 0.24456188, -5.0084376 , 12.591685  , ..., -2.447715  ,
         -2.577407  , -0.09171152],
        [ 1.9546525 , -4.852409  ,  5.4015346 , ..., -3.9360301 ,
         -1.5068203 , -0.11707431],
        ...,
        [-3.2526023 , -6.279535  ,  5.785829  , ..., -2.06605   ,
         -1.8451253 , -2.678122  ],
        [18.32295   , -5.372933  , 19.210497  , ..., -0.04520693,
          2.2998261 ,  4.34124   ],
        [ 9.742845  , -6.0795035 , 26.81919   , ..., -1.9886105 ,
         -4.526572  ,  4.77639   ]]], dtype=float32)>

In [34]:
i = mask_token_indices[0]
i
# mask 위치 

5

In [25]:
logits.shape

TensorShape([1, 9, 50265])

In [36]:
mask_token_logits = logits[0,i,:]
mask_token_logits

<tf.Tensor: shape=(50265,), dtype=float32, numpy=
array([-3.016694  , -4.8106475 ,  3.067162  , ..., -4.559116  ,
       -5.1449747 ,  0.09254533], dtype=float32)>

In [37]:
mask_token_logits.shape

TensorShape([50265])

In [39]:
top = tf.math.top_k(mask_token_logits,k=10)
top

TopKV2(values=<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([17.460817 , 16.348907 , 15.867335 , 13.808291 , 13.712693 ,
       13.596157 , 12.9192705, 12.796195 , 12.511706 , 12.448376 ],
      dtype=float32)>, indices=<tf.Tensor: shape=(10,), dtype=int32, numpy=
array([ 2674,  5863,  5548,  6543,  3366,  6813,  5440, 17771,  7476,
        7080], dtype=int32)>)

In [53]:
top.indices

<tf.Tensor: shape=(10,), dtype=int32, numpy=
array([ 2674,  5863,  5548,  3366,  6813,  6543,  7080, 15163,  7476,
       17771], dtype=int32)>

In [48]:
for i in top.indices.numpy().tolist():
    print(id2word[i])

Ġfavorite
Ġcomfort
Ġfavourite
Ġsignature
Ġdream
Ġpreferred
Ġpassion
Ġstaple
Ġeveryday
Ġbreakfast


In [50]:
sequence = f'pizza is {nlp.tokenizer.mask_token} my {nlp.tokenizer.mask_token} food.'
input_ids = tokenizer.encode(sequence,return_tensors='tf')
mask_token_indices = tf.where(input_ids[0] == tokenizer.mask_token_id)
mask_token_indices = tf.squeeze(mask_token_indices).numpy().tolist()
mask_token_indices

[4, 6]

In [51]:
result = model(input_ids)
logits = result.logits
logits.shape

TensorShape([1, 10, 50265])

In [52]:
for i in mask_token_indices:
    print(f'--{i}--')
    mask_token_logits = logits[0,i,:]
    top = tf.math.top_k(mask_token_logits,k=10)
    for i in top.indices.numpy().tolist():
        print(id2word[i])

--4--
Ġprobably
Ġdefinitely
Ġjust
Ġnow
Ġalways
Ġnot
Ġbasically
Ġliterally
Ġactually
Ġlike
--6--
Ġfavorite
Ġcomfort
Ġfavourite
Ġdream
Ġpreferred
Ġsignature
Ġbreakfast
Ġjunk
Ġeveryday
Ġstaple
